In [1]:
!jt -t oceans16 -T -N
#!jt -l

/bin/bash: line 1: jt: command not found


In [2]:
nEvents = int(50000)
nThread =8
Ecm= 200

#opts = [3,7,8,9]
opts = [1,7,8]

In [3]:
import sys
cfg = open("/home/yoren/bnl/pythia8310/examples/Makefile.inc")
lib = "/home/yoren/bnl/pythia8310/lib"
for line in cfg:
    if line.startswith("PREFIX_LIB="): lib = line[11:-1]; break
sys.path.insert(0, lib)

options = ['SoftQCD:all=on', 'SoftQCD:inelastic=on', 'HardQCD:allwdi=on', 'HardQCD:all=on', #0-3
           'HardQCD:onlydi=on', 'PromptPhoton:all=on', 'PhaseSpace:Q2Min=200.', 'PDF:pSet=8',#4-7
           'MultipartonInteractions:Kfactor=4', 'PhaseSpace:pTHatMinDiverge=20',#8,9
           'PromptPhoton:all=on', 'PromptPhoton:qg2qgamma=on', 'PromptPhoton:qqbar2ggamma=on']

! which root
import ROOT
%jsroot on

/home/yoren/bnl/ROOT/install/bin/root
Welcome to JupyROOT 6.29/01


In [4]:
import math

def angle(vector1, vector2):
    x1, y1, z1 = vector1[1], vector1[2], vector1[3]
    x2, y2, z2 = vector2[1], vector2[2], vector2[3]
    inner_product = x1*x2 + y1*y2 + z1*z2
    len1 = math.hypot(x1, y1, z1)
    len2 = math.hypot(x2, y2, z2)
    #print(inner_product,len1,len2)
    rel = inner_product/(len1*len2)
    if abs(rel)>1:
        rel = 1
    return math.acos(rel)

veto_angle = math.atan(20./522)
print(veto_angle)

0.03829544464261488


In [5]:
def inv_mass(vector1, vector2):
    e1, px1, py1, pz1 = vector1[0], vector1[1], vector1[2], vector1[3]
    e2, px2, py2, pz2 = vector2[0], vector2[1], vector2[2], vector2[3]
    m2 = (e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2
    if m2<0: m2=0
    return math.sqrt(m2)

In [6]:
import random

In [7]:
# Import the Pythia module.
import pythia8

def pythia8_runner(seed, nEvents_tred, target =2212, projectile=2212):
    #print("YOLO"*100,seed,projectile,target)
    
    hist_Nch = ROOT.TH1D("hist_Nch","hist_Nch",100,0,100)
    hist_Nch_mid = ROOT.TH2D("hist_Nch_mid","hist_Nch_mid",100,0,100,100,0,100)
    hist_part = ROOT.TH3D("hist_part","hist_part",40,0,20,5,0,5,100,0,100)
    hist_inv_mass = ROOT.TH1D("hist_inv_mass","hist_inv_mass",1000,0,1)

    pythia = pythia8.Pythia()
    pythia.readString(f"Beams:eCM = {Ecm}")
    for iopt in opts:
        pythia.readString(options[iopt])    
    pythia.readString("Random:setSeed = on")
    pythia.readString(f"Random:seed = {seed}")
    pythia.readString(f"Beams:idA = {projectile}") # The coper ion
    pythia.readString(f"Beams:idB = {target}") # The lead ions.
    pythia.readString("HeavyIon:SigFitNGen = 0")
    pythia.readString("HeavyIon:SigFitDefPar = 9.82,1.69,0.29,0.0,0.0,0.0,0.0,0.0")

    pythia.init()
        # Begin event loop. Generate event. Skip if error. List first one.
    for iEvent in range(0, int(nEvents_tred)):
        if iEvent%1000 == 0: print(iEvent, seed)
        if not pythia.next(): continue
        
        info = pythia.infoPython()
        
        weight_pythia = info.weight() 
                       
        # Find number of all final charged particles and fill histogram.
        nCharged = 0
        nCharged_mid = 0
        charged_hadrons_p=[]
        photons=[]
        for pythia_event in pythia.event:
            if pythia_event.isFinal() and pythia_event.isCharged():
                if 0 < abs(pythia_event.eta()) < 1: nCharged += 1
                if abs(pythia_event.eta()) < 0.5 : nCharged_mid +=1
                if abs(pythia_event.eta()) < 0.35 : #and random.random()<0.5
                    charged_hadrons_p.append(pythia_event.p())        
            if abs(pythia_event.eta()) < 0.35 and pythia_event.id()==22 and pythia_event.isFinal():
                if  random.random()<0.5:
                    #print(pythia.event[pythia_event.mother1()].id())
                    photons.append(pythia_event.p())

        hist_Nch.Fill(nCharged,weight_pythia)
        hist_Nch_mid.Fill(nCharged_mid,nCharged,weight_pythia)
        
        for pythia_event in pythia.event:  
            if abs(pythia_event.eta()) < 0.35:
                if pythia_event.id() == 221:   
                    daughter1 = pythia.event[pythia_event.daughter1()]
                    daughter2 = pythia.event[pythia_event.daughter2()]
                    if daughter1.id()!=22 or daughter2.id()!=22 or abs(daughter1.eta())>0.35 or abs(daughter2.eta())>0.35: continue
                    hist_part.Fill(pythia_event.pT(),0,nCharged,weight_pythia)
                    eta_ph1 = daughter1.p()
                    eta_ph2 = daughter2.p()
                    veto = False
                    for hadron_p in charged_hadrons_p:
                        ang1 = angle(eta_ph1,hadron_p)
                        ang2 = angle(eta_ph2,hadron_p)
                        if ang1<veto_angle or ang2<veto_angle: 
                            #print(ang,veto_angle)
                            veto = True
                    if veto: continue
                    hist_part.Fill(pythia_event.pT(),1,nCharged,weight_pythia)
                    veto_pi0 = False
                    for photon in photons:
                        mass1 = inv_mass(daughter1.p(),photon)
                        mass2 = inv_mass(daughter2.p(),photon)
                        hist_inv_mass.Fill(mass1)
                        hist_inv_mass.Fill(mass2)
                        #print(mass1)
                        if 0.125<mass1<0.145 or 0.125<mass2<0.145: 
                            #print(mass1, mass2)
                            veto_pi0 = True
                    if veto_pi0: continue
                    hist_part.Fill(pythia_event.pT(),2,nCharged,weight_pythia)
            
            
# End of event loop. Statistics. Histogram. Done.
    pythia.stat()
    return [hist_Nch, hist_part, hist_inv_mass, hist_Nch_mid]

In [8]:
#second(156,5000)
import multiprocess as mp

Ntr = nThread
Nev = nEvents
seed0 = 15156
seed_iter = 43324

pn_ids = [int(2212), int(2112)]

projectile_params = [pn_ids[int(1.34*i/(Ntr))] for i in range(Ntr)]
target_params = [pn_ids[min(int(4*i/Ntr), 1)] for i in range(Ntr)]
print(projectile_params, target_params)

pool = mp.Pool(Ntr)
hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, 2212, 2212) for i in range(Ntr)])
#hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, 1000020040, 1000791970) for i in range(Ntr)]) #1000020030
#hist_array = pool.starmap(pythia8_runner, [(seed0+i*seed_iter, Nev, projectile_params[i], target_params[i]) for i in range(Ntr)])
pool.close()

[2212, 2212, 2212, 2212, 2212, 2212, 2112, 2112] [2212, 2212, 2112, 2112, 2112, 2112, 2112, 2112]
0 145128
0 318424
000   
5848010180415156

0 0275100
 188452
0 231776
1000 145128
1000 101804
1000 15156
1000 58480
1000 318424
1000 275100
1000 188452
1000 231776
2000 58480
2000 188452
2000 145128
2000 318424
2000 101804
2000 15156
2000 275100
2000 231776
3000 145128
3000 15156
3000 101804
3000 318424
3000 188452
3000 275100
3000 58480
3000 231776
4000 145128
4000 188452
4000 15156
4000 101804
4000 318424
4000 275100
4000 231776
4000 58480
5000 145128
5000 188452
5000 275100
5000 101804
5000 15156
5000 318424
5000 231776
5000 58480
6000 145128
6000 188452
6000 101804
6000 15156
6000 275100
6000 231776
6000 58480
6000 318424
7000 145128
7000 188452
7000 231776
7000 101804
7000 318424
7000 58480
7000 275100
7000 15156
8000 145128
8000 188452
8000 231776
8000 101804
8000 318424
8000 58480
8000 15156
8000 275100
9000 188452
9000 145128
9000 231776
9000 101804
9000 318424
9000 58480
9000 1515

In [9]:
hist_Nch = hist_array[0][0]
for i in range(1, Ntr):
    hist_Nch.Add(hist_array[i][0])
hist_pt = hist_array[0][1]
for i in range(1, Ntr):
    hist_pt.Add(hist_array[i][1])
hist_imass = hist_array[0][2]
for i in range(1, Ntr):
    hist_imass.Add(hist_array[i][2])
hist_Nch_mid = hist_array[0][3]
for i in range(1, Ntr):
    hist_Nch_mid.Add(hist_array[i][3])

In [10]:
c1 = ROOT.TCanvas("c1","c1",1080,540)
total = hist_Nch.GetSumOfWeights()
boarders = [1,2,3,4,100]
sums = []
for jj in range(1,4):
    local_var = 0
    for ii in range(hist_Nch.GetNbinsX()+1,1,-1):
        local_var += hist_Nch.GetBinContent(ii)
        if local_var >= total/5*jj: 
            boarders[4-jj]=hist_Nch.FindBin(ii)
            sums.append(local_var-sum(sums))
            break
sums.append(total-sum(sums))
print(boarders, sums)    
hist_Nch.Draw()
c1.Draw()

[1, 6, 11, 21, 100] [83809.0, 76884.0, 96775.0, 142532.0]


In [11]:
c3 = ROOT.TCanvas("c3","c3",1080,540)
hist_Nch_mid.Draw()
c3.Draw()

nch = []
for ii in range(4):
    local_proj = hist_Nch_mid.ProjectionX(f"proj{ii}",boarders[ii],boarders[ii+1]-1)
    nch.append(local_proj.GetMean())

print(nch)

[1.1739865168001626, 3.7706414315870274, 7.39010185133082, 15.215780474422285]


In [12]:
c4 = ROOT.TCanvas("c4","c4",540,540)

hist_eta_orig = hist_pt.ProjectionX("hist_eta_orig01",1,1,1,100)
hist_eta_veto1 = hist_pt.ProjectionX("hist_eta_veto10",2,2,1,100)
hist_eta_veto1.Divide(hist_eta_orig)
for ibin in range(1,hist_eta_veto1.GetNbinsX()+1):
    hist_eta_veto1.SetBinError(ibin, 1.42/(hist_eta_orig.GetBinContent(ibin)+0.1)**0.5*hist_eta_veto1.GetBinContent(ibin))
hist_eta_veto1.SetLineColor(1)
hist_eta_veto1.Draw()
hist_eta_veto1.Fit("pol1","","",0,20)
fit = hist_eta_veto1.GetFunction("pol1")
fit.SetLineColor(1)

hist_eta_orig_arr=[]
hist_eta_veto_ARR=[]
fits=[]
for ii in range(4):
    hist_eta_orig_arr.append(hist_pt.ProjectionX(f"hist_eta_orig{ii}",1,1,boarders[3-ii],boarders[4-ii]-1))
    hist_eta_veto_ARR.append(hist_pt.ProjectionX(f"hist_eta_veto{ii}",2,2,boarders[3-ii],boarders[4-ii]-1))
    hist_eta_veto_ARR[-1].Divide(hist_eta_orig_arr[-1])
    for ibin in range(1,hist_eta_veto_ARR[-1].GetNbinsX()+1):
        hist_eta_veto_ARR[-1].SetBinError(ibin, 1.42/(hist_eta_orig_arr[-1].GetBinContent(ibin)+0.1)**0.5*hist_eta_veto_ARR[-1].GetBinContent(ibin))
    hist_eta_veto_ARR[-1].SetLineColor(2+ii)
    hist_eta_veto_ARR[-1].Draw("same")
    hist_eta_veto_ARR[-1].Fit("pol1","","",0,20)
    fits.append(hist_eta_veto_ARR[-1].GetFunction("pol1"))
    fits[-1].SetLineColor(2+ii)


c4.Print("heau_veto_new.png")

****************************************
Minimizer is Linear / Migrad
Chi2                      =    0.0639526
NDf                       =            9
p0                        =     0.992444   +/-   0.0297437   
p1                        =  -0.00809605   +/-   0.0271234   
****************************************
Minimizer is Linear / Migrad
Chi2                      =    0.0542929
NDf                       =            8
p0                        =     0.988659   +/-   0.0405511   
p1                        =  -0.00747929   +/-   0.0331701   
****************************************
Minimizer is Linear / Migrad
Chi2                      =    0.0342217
NDf                       =            8
p0                        =     0.992191   +/-   0.0624284   
p1                        =  -0.00373802   +/-   0.0615217   
****************************************
Minimizer is Linear / Migrad
Chi2                      =   0.00861832
NDf                       =            4
p0                  

Info in <TCanvas::Print>: png file heau_veto_new.png has been created


In [13]:
c4.Draw()

In [14]:
c2 = ROOT.TCanvas("c2","c2",1080,540)

c2.Divide(2,1)
c2.cd(1)
hist_eta_orig = hist_pt.ProjectionX("hist_eta_orig",1,1,1,100)
hist_eta_orig.SetLineColor(4)
hist_eta_orig.Draw()
hist_eta_veto = hist_pt.ProjectionX("hist_eta_veto",2,2,1,100)
hist_eta_veto.SetLineColor(6)
hist_eta_veto.Draw("same")
hist_eta_veto_pi0 = hist_pt.ProjectionX("hist_eta_veto_pi0",3,3,1,100)
hist_eta_veto_pi0.SetLineColor(2)
hist_eta_veto_pi0.Draw("same")
c2.cd(2)
hist_eta_veto1 = hist_pt.ProjectionX("hist_eta_veto1",2,2,1,100)
hist_eta_veto1.Divide(hist_eta_orig)
hist_eta_veto1.SetLineColor(6)
hist_eta_veto_pi01 = hist_pt.ProjectionX("hist_eta_veto_pi01",3,3,1,100)
hist_eta_veto_pi01.Divide(hist_eta_orig)
hist_eta_veto_pi01.SetLineColor(2)
hist_eta_veto1.Draw()
hist_eta_veto_pi01.Draw("same")

c2.Print("heau_veto.png")

c2.Draw()

Info in <TCanvas::Print>: png file heau_veto.png has been created


In [15]:
c5 = ROOT.TCanvas("c5","c5",1080,540)
c5.Divide(2,1)
c5.cd(1)
hist_pt.Draw()
proj=hist_pt.ProjectionZ("kek",1,-1,1,-1)
c5.cd(2)
proj.Draw()
c5.Draw()


In [16]:
output_file = ROOT.TFile.Open("NICA/HeAu_veto.root","RECREATE")
hist_pt.Write()
hist_Nch.Write()
hist_imass.Write()
output_file.Close()